In [2]:
import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [3]:
text = """artificial intelligence is transforming modern society.
it is used in healthcare finance education and transportation.
machine learning allows systems to improve automatically with experience.
data plays a critical role in training intelligent systems.
large datasets help models learn complex patterns.
deep learning uses multi layer neural networks.
neural networks are inspired by biological neurons.
each neuron processes input and produces an output.
training a neural network requires optimization techniques.
gradient descent minimizes the loss function.

natural language processing helps computers understand human language.
text generation is a key task in nlp.
language models predict the next word or character.
recurrent neural networks handle sequential data.
lstm and gru models address long term dependency problems.
however rnn based models are slow for long sequences.

transformer models changed the field of nlp.
they rely on self attention mechanisms.
attention allows the model to focus on relevant context.
transformers process data in parallel.
this makes training faster and more efficient.
modern language models are based on transformers.

education is being improved using artificial intelligence.
intelligent tutoring systems personalize learning.
automated grading saves time for teachers.
online education platforms use recommendation systems.
technology enhances the quality of learning experiences.

ethical considerations are important in artificial intelligence.
fairness transparency and accountability must be ensured.
ai systems should be designed responsibly.
data privacy and security are major concerns.
researchers continue to improve ai safety.

text generation models can create stories poems and articles.
they are used in chatbots virtual assistants and content creation.
generated text should be meaningful and coherent.
evaluation of text generation is challenging.
human judgement is often required.

continuous learning is essential in the field of ai.
research and innovation drive technological progress.
students should build strong foundations in mathematics.
programming skills are important for ai engineers.
practical experimentation enhances understanding.
"""

In [55]:
def preprocessed(text):
    text.lower()
    chars = sorted(list(set(text)))
    vocab_size = len(chars)

    char2idx = {ch:i for i,ch in enumerate(chars)}
    idx2char = {i:ch for ch,i in char2idx.items()}

    encoded = [char2idx[ch] for ch in text]

    return encoded, char2idx, idx2char, vocab_size


In [56]:
def create_seq(encoded_txt, seq_len):
    x =[]
    y =[]

    for i in range(len(encoded_txt) - seq_len):
        x.append( encoded_txt[i:i+seq_len] )
        y.append( encoded_txt[i+seq_len] )

    x = torch.tensor(x)
    y = torch.tensor(y)

    return x,y
    

In [57]:
seq_len = 5
encoded, char2idx, idx2char, vocab_size = preprocessed(text)
x,y = create_seq(encoded, seq_len)
print(len(x))
print(vocab_size)
x[:3]

2210
29


tensor([[ 3, 20, 22, 11,  8],
        [20, 22, 11,  8, 11],
        [22, 11,  8, 11,  5]])

In [58]:
class MannualRNN(nn.Module):
    def __init__(self, hidden_size, vocab_size):
        super(MannualRNN, self).__init__()

        self.hidden_size = hidden_size
        self.tanh = nn.Tanh()

        self.emb = nn.Embedding(vocab_size,hidden_size)
        self.wxh = nn.Linear(hidden_size, hidden_size)
        self.whh = nn.Linear(hidden_size, hidden_size)
        self.why = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        h = torch.zeros(1, self.hidden_size)

        for t in range(x.size(0)):
            x_t = self.emb(x[t]).unsqueeze(0)
            h = self.tanh(self.wxh(x_t) + self.whh(h))

        out = self.why(h)
        return out

In [59]:
hidden_size = 32
model = MannualRNN(hidden_size, vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr =0.01)

In [60]:
# len(x)
print(y[0])
print(len(x), len(y))

tensor(11)
2210 2210


In [61]:
epochs = 50
for e in range(epochs):
    t_loss =0
    for i in range(len(x)):
        seq= x[i]
        target = y[i].unsqueeze(0)
        optimizer.zero_grad()
        predic_output = model(seq)
        loss = criterion(predic_output, target)
        loss.backward()
        optimizer.step()

        t_loss += loss.item()

    avg_loss = t_loss/len(x)

    print(f'epoch {e+1}: loss {avg_loss:.4f}')

epoch 1: loss 2.7836
epoch 2: loss 2.4280
epoch 3: loss 2.2828
epoch 4: loss 2.1705
epoch 5: loss 2.0752
epoch 6: loss 1.9937
epoch 7: loss 1.9209
epoch 8: loss 1.8549
epoch 9: loss 1.7954
epoch 10: loss 1.7405
epoch 11: loss 1.6910
epoch 12: loss 1.6478
epoch 13: loss 1.6083
epoch 14: loss 1.5700
epoch 15: loss 1.5413
epoch 16: loss 1.5231
epoch 17: loss 1.4999
epoch 18: loss 1.4830
epoch 19: loss 1.4664
epoch 20: loss 1.4429
epoch 21: loss 1.4328
epoch 22: loss 1.4427
epoch 23: loss 1.4312
epoch 24: loss 1.4281
epoch 25: loss 1.4182
epoch 26: loss 1.4207
epoch 27: loss 1.4134
epoch 28: loss 1.4101
epoch 29: loss 1.4378
epoch 30: loss 1.4542
epoch 31: loss 1.3959
epoch 32: loss 1.3852
epoch 33: loss 1.4002
epoch 34: loss 1.3608
epoch 35: loss 1.3617
epoch 36: loss 1.3896
epoch 37: loss 1.4042
epoch 38: loss 1.4262
epoch 39: loss 1.3991
epoch 40: loss 1.4179
epoch 41: loss 1.3645
epoch 42: loss 1.3888
epoch 43: loss 1.3498
epoch 44: loss 1.3803
epoch 45: loss 1.3834
epoch 46: loss 1.35

In [62]:
model.eval()
def generate(model, base_text, length=50):
    base_text = base_text.lower()
    text = base_text

    for _ in range(length):
        seq = torch.tensor([char2idx[c] for c in base_text])
        with torch.no_grad():
            output = model(seq)
            probs = torch.softmax(output, dim=1)
            next_idx = torch.multinomial(probs, 1).item()
        
        next_char = idx2char[next_idx]
        text += next_char
        
        base_text = text[-seq_len:]
    
    return text

In [63]:
print(vocab_size)
print(len(idx2char))


29
29


In [64]:
print(generate(model, "artificial ", 40))


artificial is sovacy are shthand utual in nuractima
